Tutorial to forecast uni-variate time-series With XGBoost

In this article, I shall be providing a tutorial on how to build an XGBoost model to handle a uni-variate time-series electricity dataset. The goal is to create a model that will allow us to confidently predict future electricity consumption in China. By the end of this article, you will be able to understand why we should use XGBoost, and how to write code to implement the model in Jupyter Notebook. 

Download Data
This data has been downloaded originally from the online database created and maintained by the Research Department at the Federal Reserve Bank of St. Louis (FRED). If you wish to follow along, you can download the dataset hosted on my Github.

XGBoost 
https://xgboost.readthedocs.io/en/latest/
XGBoost is an optimized distributed gradient boosting library designed to have high computation speed & performance. In Python, the XGBoost library gives you a supervised machine learning model that follows the Gradient Boosting framework. It uses a parallel tree boosting algorithm to create forecasts. The XGBoost package in Python can handle LIBSVM text format files, CSV files, Numpy 2D arrays, SciPy 2D sparse arrays, cuDF DataFrames and Pandas DataFrames. In this example, we will be using a CSV file. You can install XGBoost by typing the following in your command prompt: 

pip install xgboost

Importing Dependencies 
We begin by opening the Jupyter notebook and typing pip install module. This "module" is being invoked so that we can use it in our code. The modules that will be used in this tutorial are NumPy, Pandas, Plotly, SKlearn and XGBoost modules.

In [21]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objs as go
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools
username = 'xyz' # Add your username
api_key = 'OOs2fe0fSrV1goPhnT5p' # Add your api key
chart_studio.tools.set_credentials_file(username = username, api_key = api_key)

Loading Data

We use Pandas to import the CSV file with headers. Let us print the first 10 elements of our dataset to see what's in it!

In [22]:
#Read the dataset and print the first 10 elements of the dataset
data = pd.read_csv('CHNPRENEL01MLM.csv')
data.head()
data.head(10)

,DATE,Electricity
0,2000-01-01,105230.0
1,2000-02-01,95740.0
2,2000-03-01,104390.0
3,2000-04-01,103720.0
4,2000-05-01,109780.0
5,2000-06-01,109810.0
6,2000-07-01,119370.0
7,2000-08-01,117410.0
8,2000-09-01,107900.0
9,2000-10-01,108970.0


As you can see this dataset contain electricity production from China. These numbers are in Gigawatt hours and are not adjusted for seasonal variation. Now, let us check the data types of our columns present in the Pandas DataFrame 'data' using built-in property 'dtypes'. 

In [23]:
data.dtypes

DATE            object
Electricity    float64
dtype: object

We can clearly see that this dataframe contains a column 'Date' that needs to be changed into Pandas 'datetime' format. 

In [24]:
data['DATE'] = pd.to_datetime(data.DATE)

In this step, we will extract the "Year" and "Month" column from the "Date" column using the built-in property "DatetimeIndex". We have to complete this step to make the DataFrame readable for "DMatrix" data structure that XGBoost requires for functioning properly in Python. 

In [25]:
data['Year'] = pd.DatetimeIndex(data['DATE']).year
data['Month'] = pd.DatetimeIndex(data['DATE']).month

In this step, we will quickly check to see the shape of our DataFrame. We currently have 5 columns and 3,265 rows of data.

In [26]:
# check the number of rows and columns in DataFrame
print(data.shape)

(256, 4)


DMatrix Data Structure

In order to run the dataset with XGBoost, it requires that the data be fed into DMatrix data structure. It optimizes both memory efficiency and training speed of the XGBoost model.

We will now separate the target variable - 'Electricity' and rest of the variables using .iloc to subset the data . This needs to be done to make our data compatible for DMatrix's structure. 

In [27]:
X, y =  data.loc[:,['Month', 'Year']].values, data.loc[:,'Electricity'].values
data_dmatrix = xgb.DMatrix(X,label=y)

Train-Test Split

Next, we will create a training and testing dataset using the train_test_split function from sklearn's model_selection module. The training set will contain a known output and the model will learn on this data while the testing set will be used to test our model's prediction.

The training data will account for 80% oF electricity data while the testing data will account for the rest. We will use "random_state" property to make sure that our splitted datasets do not change every time we run the function. 

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

XGBRegressor

For this particular example, we'll use the XGBRegressor class with hyperparameters passed as arguments to optimize our model. We will be choosing the 'reg:squarederror' objective loss function for predicting electricity values. Other loss functions included in the package for solving regression problems are 'reg:squaredlogerror', 'reg:logistic', 'reg:pseudohubererror', 'reg:gamma', and 'reg:tweedie'. However, none of these functions gave me results that were as good as the results I got by using 'reg:squarederror'. Other hyperparameters used here are: 

n_estimators – This parameter defines the number of boosting rounds.
max_depth - This parameter identifies the maximum tree depth for base learners.
learning_rate - This parameter indicates the boosting learning rate (xgb’s “eta”)
subsample - This parameter defines the subsample ratio of the training dataset.
colsample_bytree - This parameter provides the subsample ratio of columns when constructing each tree.


In [29]:
reg_mod = xgb.XGBRegressor(objective='reg:squarederror',
    n_estimators=1000,
    learning_rate=0.10,
    subsample=0.5,
    colsample_bytree=1, 
    max_depth=5,
)
reg_mod.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.5,
             tree_method='exact', validate_parameters=1, verbosity=None)

Make Predictions

Next, we will use the XGBRegressor to fit the training data & make prediction on the testing data. 

In [30]:
reg_mod.fit(X_train,y_train)

predictions = reg_mod.predict(X_test)

Accuracy Metrics

After we have built a model, we can check its accuracy by calculating the RMSE score and the R-squared score.
RMSE (Root Mean Squared Error) of an estimator of a population parameter is the square root of the mean square error (MSE). The mean square error is defined as the expected value of the square of the difference between the estimator and the parameter. It is the sum of variance and squared Bias.
R-squared value, denoted by R2 is a statistical measure that calculates the proportion of variation in the dependent variable that can be attributed to the independent variable.

In [31]:
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print("RMSE: %f" % (rmse))

RMSE: 13379.472413


In [32]:
from sklearn.metrics import r2_score
r2 = np.sqrt(r2_score(y_test, predictions))
print("R_Squared Score : %f" % (r2))

R_Squared Score : 0.997134


The R-squared score of 99.7% and RMSE of 13,379 indicates that we have a decent model that can be used to predict values into the future. 

Test vs Predicted Values

Finally, we'll visualize the original and predicted test data in a plot using the Plotly library. To accomplish this, I will create an empty dataframe and assign the tested and predicted values into "result" dataframe. 

In [33]:
# Creating an empty Dataframe with column names only
result = pd.DataFrame(columns=['Time', 'Test', 'Predicted'])
result['Time'] = pd.date_range(start='1/1/2000', periods=52, freq='M')
result['Test'] = y_test
result['Predicted'] = predictions
#Using Plotly to build the graph
fig = go.Figure()
fig.add_trace(go.Scatter(x=result['Time'], y=result['Test'],
                    mode='lines',
                    name='Test'))
fig.add_trace(go.Scatter(x=result['Time'], y=result['Predicted'],
                    mode='lines',
                    name='Predicted'))

# Edit the layout
fig.update_layout(title='Test vs Predicted Values',
                   xaxis_title='Months',
                   yaxis_title='Electricity (GWh)',
                   template='gridon')

fig.show()

Visualize Forecasted Values
Finally, the last piece of code will print the electricity consumption values until 2026.

In [34]:
df=pd.DataFrame(predictions, columns=['pred']) 
df['date'] = pd.date_range(start='2/28/2021', periods=len(df), freq='M')
fig = px.line(df, x="date", y="pred")
# Edit the layout
fig.update_layout(title='China Electricity Consumption Forecast',
                   xaxis_title='Date',
                   yaxis_title='Electricity - GWh',
                   template='gridon')
fig.show()

Conclusion
This was a step-by-step tutorial for anybody who is starting with XGBoost and requires to use this supervised machine learning approach to solve regression problems. We covered how to mould your dataset to fit the DMatrix data structure inherent to the XGBoost model. Moreover, you applied the various parameters used to optimize your model using XGBRegressor class. 
Finally, we visualized the test, predicted and forecasted electricity values obtained from the trained XGBoost model.